In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import pickle

In [2]:
def a(*args):
    if len(args) == 1: 
        return np.array(args[0])
    else:
        return np.array(args)

def relu(x):
    if x > 0:
        return x
    else:
        return 0
    
def reluderiv(x):
    if x > 0:
        return 1
    else:
        return 0
    
def no(x):
    return x

In [3]:
def propagate(input,weights,biases,activation):
    output = np.dot(input,weights) + biases
    for i in range(len(output)):
        output[i] = activation(output[i])

    return output

In [4]:
def rand(size):
    return np.random.normal(size=size)

In [5]:
class network:
    def __init__(self, layers):
        self.layers = layers
        self.weights = []
        self.biases = []

        for i in range(len(layers)-1):
            self.biases.append(rand(layers[i+1]))

        for i in range(len(layers)-1):
            self.weights.append(rand((layers[i],layers[i+1])))

    def saveModel(self,filename):
        with open(filename,'wb+') as f:
            f.write(pickle.dumps(self.layers) + b"\n")
            f.write(pickle.dumps(self.weights) + b"\n")
            f.write(pickle.dumps(self.biases) + b"\n")

    def loadModel(self,filename):
        with open(filename,"rb") as f:
            self.layers = pickle.loads(f.readline()[:-1])
            self.weights = pickle.loads(f.readline()[:-1])
            self.biases = pickle.loads(f.readline()[:-1])

    def propagate(self,inp):
        activations = [inp]
        values = [inp]
        current = inp
        for i in range(len(self.layers)-1):
            current = propagate(current,self.weights[i], self.biases[i],no)

            values.append(current.copy())
            for i in range(len(current)):
                current[i] = relu(current[i])
            activations.append(current.copy())

        return current,activations,values

    def calculateErrors(self,inp,target):
        output,activations,values = self.propagate(inp)
        cost = sum((output-target)**2)
        errors = [(output-target) * a([reluderiv(values[-1][i]) for i in range(len(values[-1]))])]


        for i in range(len(self.layers)-1):

            weights = self.weights[-(i+1)].T
            previousErrors = errors[0]
            derivActivation = a([reluderiv(values[-(i+2)][j]) for j in range(len(values[-(i+2)]))])

            errors.insert(0,(np.dot(previousErrors,weights) * derivActivation))

        print(errors)
        print(values)

        biasDeriv = errors.copy()

        weightDeriv = []

        for i in range(len(self.layers)-1):
            weightDeriv.append(np.zeros((self.layers[i],self.layers[i+1])))
            
            for fn in range(self.layers[i]):
                for tn in range(self.layers[i+1]):
                    weightDeriv[-1][fn][tn] = activations[i][fn] * errors[i+1][tn]

        return errors,biasDeriv,weightDeriv
    
    def tweakNetwork(self, learnFactor, biasDeriv, weightDeriv):
        for i in range(len(self.layers)-1):
            self.weights[i] -= weightDeriv[i]*learnFactor

        #print(biasDeriv)
        #print(self.biases)

        for i in range(len(self.layers)-1):
            self.biases[i] += learnFactor * biasDeriv[1:][i]

In [6]:
n = network([])

n.loadModel("test2.net")

print(n.weights)
print(n.biases)

print(n.propagate(1))

print(n.calculateErrors([1],[2]))

[array([[0.14974665]])]
[array([0.06429983])]
(array([[0.21404648]]), [1, array([[0.21404648]])], [1, array([[0.21404648]])])
[array([-0.26744055]), array([-1.78595352])]
[[1], array([0.21404648])]
([array([-0.26744055]), array([-1.78595352])], [array([-0.26744055]), array([-1.78595352])], [array([[-1.78595352]])])
